<a href="https://colab.research.google.com/github/tnc-br/ddf-isoscapes/blob/kriging_ensemble_data/dnn/kriging_ensemble_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
MEAN_PREDICTED_NAME = 'ordinary_kriging_linear_d18O_predicted_mean' #@param
VAR_PREDICTED_NAME = 'ordinary_kriging_linear_d18O_predicted_variance' #@param

INPUT_PATH = 'canonical/uc_davis_2023_08_26_isoscapes_test_fixed_grouped.csv' #@param

MEANS_ISOSCAPE_PATH = 'canonical/uc_davis_d18O_cel_ordinary_linear_fixed_grouped_means.tiff' #@param
VARS_ISOSCAPE_PATH = 'canonical/uc_davis_d18O_cel_ordinary_linear_fixed_grouped_vars.tiff' #@param

OUTPUT_PATH = 'kriging_ensemble_experiment/uc_davis_2023_08_26_isoscapes_test_fixed_grouped_ordinary_kriging_linear.csv' #@param


In [2]:
#@title Imports and modules.
%pip install opencv-python
%pip install matplotlib
%pip install pandas

from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
import matplotlib.animation as animation
from matplotlib import rc
from typing import List
from numpy.random import MT19937, RandomState, SeedSequence
import pandas as pd
from tqdm import tqdm
from io import StringIO
import xgboost as xgb
import os
import math
import glob

rc('animation', html='jshtml')

import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
# ddfimport.ddf_source_control_pane()
ddfimport.ddf_import_common()

Cloning into 'ddf_common_stub'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 4), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 5.50 KiB | 5.50 MiB/s, done.
Resolving deltas: 100% (4/4), done.
executing checkout_branch ...
b''
main branch checked out as readonly. You may now use ddf_common imports


In [3]:
import dataset

In [4]:
import raster

In [6]:
raster.mount_gdrive()

Mounted at /content/gdrive


In [28]:
df = pd.read_csv(raster.get_sample_db_path(INPUT_PATH), index_col=0)

In [13]:
means_isoscape = raster.load_raster(raster.get_raster_path(MEANS_ISOSCAPE_PATH), use_only_band_index=0)
vars_isoscape = raster.load_raster(raster.get_raster_path(VARS_ISOSCAPE_PATH), use_only_band_index=0)

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)


In [30]:
df[MEAN_PREDICTED_NAME] = df.apply(lambda row:raster.get_data_at_coords(means_isoscape, row['long'],row['lat'],-1), axis=1)
df[VAR_PREDICTED_NAME] = df.apply(lambda row:raster.get_data_at_coords(vars_isoscape, row['long'],row['lat'],-1), axis=1)

In [31]:
df

,lat,long,VPD,RH,PET,DEM,PA,Mean Annual Temperature,Mean Annual Precipitation,Iso_Oxi_Stack_mean_TERZER,predkrig_br_lat_ISORG,isoscape_fullmodel_d18O_prec_REGRESSION,Code,Family,Origin,d18O_cel_mean,d18O_cel_variance,ordinary_kriging_linear_d18O_predicted_mean,ordinary_kriging_linear_d18O_predicted_variance
17,-6.009707,-61.868657,0.77083,0.79509,93.97500,71,1004.47662,27.20000,1996,-4.05694,-3.91077,-4.46622,mad19,myristicaceae,manicore,24.042000,0.345970,25.869753,1.374346
18,-6.009707,-61.868657,0.77083,0.79509,93.97500,71,1004.47662,27.20000,1996,-4.05694,-3.91077,-4.46622,mad20,myristicaceae,manicore,25.240000,0.035950,25.869753,1.374346
30,-6.009707,-61.868657,0.77083,0.79509,93.97500,71,1004.47662,27.20000,1996,-4.05694,-3.91077,-4.46622,mad32,myristicaceae,manicore,25.782000,0.372220,25.869753,1.374346
33,-6.009707,-61.868657,0.77083,0.79509,93.97500,71,1004.47662,27.20000,1996,-4.05694,-3.91077,-4.46622,mad35,myristicaceae,manicorÃÂ©,25.076000,0.230280,25.869753,1.374346
36,-6.009707,-61.868657,0.77083,0.79509,93.97500,71,1004.47662,27.20000,1996,-4.05694,-3.91077,-4.46622,mad4,burseraceae,manicore,25.966000,0.172480,25.869753,1.374346
60,-13.081300,-52.377100,0.99167,0.71167,105.92500,391,966.77917,25.06667,1593,-3.54571,-4.55400,-4.11961,mad589,burseraceae,tanguro,27.434000,0.501030,25.869753,1.374346
62,-13.079000,-52.386400,0.99167,0.71167,105.92500,381,967.93964,25.06667,1593,-3.54571,-4.55400,-4.11961,mad590,burseraceae,tanguro,28.156000,0.999030,25.869753,1.374346
63,-13.079000,-52.386400,0.99167,0.71167,105.92500,381,967.93964,25.06667,1593,-3.54571,-4.55400,-4.11961,mad591,fabaceae,tanguro,26.836000,0.120880,25.869753,1.374346
64,-13.079000,-52.386400,0.99167,0.71167,105.92500,381,967.93964,25.06667,1593,-3.54571,-4.55400,-4.11961,mad592,combretaceae,tanguro,28.180000,0.778250,25.869753,1.374346
65,-13.079000,-52.386400,0.99167,0.71167,105.92500,381,967.93964,25.06667,1593,-3.54571,-4.55400,-4.11961,mad593,apocynaceae,tanguro,26.834000,0.094930,25.869753,1.374346


In [32]:
df.to_csv(raster.get_sample_db_path(OUTPUT_PATH))